In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imghdr
import tensorflow as tf
from PIL import Image
import seaborn as sns

from sklearn.metrics import confusion_matrix,classification_report,roc_curve,auc
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd

from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.applications import VGG16, ResNet50V2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import plot_model

In [ ]:
# Define a function to count the number of files (assumed to be images for this context) for each subdirectory in a given directory.
# The function returns a DataFrame with these counts, indexed by a specified set name (e.g., 'train' or 'test').
def count_files_in_subdirs(directory, set_name):
    # Initialize an empty dictionary to hold the count of files for each subdirectory.
    counts = {}

    # Iterate over each item in the given directory.
    for item in os.listdir(directory):
        # Construct the full path to the item.
        item_path = os.path.join(directory, item)

        # Check if the item is a directory.
        if os.path.isdir(item_path):
            # Count the number of files in the subdirectory and add it to the dictionary.
            counts[item] = len(os.listdir(item_path))

    # Convert the counts dictionary to a DataFrame for easy viewing and analysis.
    # The index of the DataFrame is set to the provided set name.
    df = pd.DataFrame(counts, index=[set_name])
    return df

# Paths to the training and testing directories.
train_dir = '/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/train'
test_dir = '/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/test'

# Count the files in the subdirectories of the training directory and print the result.
train_count = count_files_in_subdirs(train_dir, 'train')
print(train_count)

# Count the files in the subdirectories of the testing directory and print the result.
test_count = count_files_in_subdirs(test_dir, 'test')
print(test_count)

In [ ]:
test_count.transpose().plot(kind='bar')

In [ ]:
import os
import matplotlib.pyplot as plt

diseases = os.listdir(train_dir)  # List all disease folders in the training directory
plt.figure(figsize=(15, 10))

for i, disease in enumerate(diseases[:4], 1):  # Limit to 4 diseases for visualization
    folder = os.path.join(train_dir, disease)
    files = os.listdir(folder)

    # Ensure folder has enough images
    if len(files) > 42:
        img_path = os.path.join(folder, files[42])  # Access the 42nd image
        img = plt.imread(img_path)
        plt.subplot(2, 2, i)  # Create a 2x2 grid for visualization
        plt.imshow(img,cmap='gray')
        plt.title(f'Disease: {disease}')
        plt.axis('off')
    else:
        print(f"Not enough images in {disease} folder.")

plt.tight_layout()
plt.show()

In [ ]:
def plot_img_from_directory(directory,set_name,num_images=9):
    img_filenames=os.listdir(directory)
    if len(img_filenames)<num_images:
        print(f"only found {len(img_filenames)} in directory {directory} display them all")
        num_images=len(img_filenmaes)
    selected_img=random.sample(img_filenames,num_images)
    fig,axis=plt.subplots(3,3,figsize=(5,5))
    axis=axis.ravel()
    for i,img_id in enumerate(selected_img):
        img_path=os.path.join(directory,img_id)
        img=load_img(img_path)
        axis[i].imshow(img)
        axis[i].set_title(f'Image--{set_name}')
        axis[i].axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
normal_dir_path='/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/train/Normal'
plot_img_from_directory(normal_dir_path,set_name='normal')

In [ ]:
normal_dir_path='/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/train/Tuberculosis'
plot_img_from_directory(normal_dir_path,set_name='Tuberculosis')

In [ ]:
normal_dir_path='/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/train/Viral Pneumonia'
plot_img_from_directory(normal_dir_path,set_name='Pneumonia')

In [ ]:
img_path='/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/train/Bacterial Pneumonia/158.jpeg'
import cv2
img=plt.imread(img_path)
print(img.shape)

In [ ]:
img_width=256
img_height=256
batch_size=32
num_classes=5

In [ ]:
train_data_dir='/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/train'
test_data_dir='/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/test'
val_data_dir='/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/val'

In [ ]:
data_generator=ImageDataGenerator(rescale=1./255,validation_split=0.2)

train_generator=data_generator.flow_from_directory(
    train_data_dir,
    target_size=(img_width,img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
)
validation_generator=data_generator.flow_from_directory(
    val_data_dir,
    target_size=(img_width,img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
)

test_generator=data_generator.flow_from_directory(
    test_data_dir,
    target_size=(img_width,img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',

)

In [ ]:
validation_generator


In [ ]:
# aceess the indices
train_data_indices=train_generator.class_indices
print(train_data_indices)
test_data_indices=test_generator.class_indices
print(test_data_indices)
validation_data_indices=validation_generator.class_indices
print(validation_data_indices)

In [ ]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,1)))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5,activation='softmax'))

In [ ]:
# Compiling the model
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
plot_model(model,show_shapes=True,show_layer_names=True)

In [ ]:
# Callback for early stopping
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # Switch to a different metric if needed
    factor=0.2,
    patience=6,
    verbose=1,
    min_delta=0.0001
)


In [ ]:
callbacks=[earlystop,reduce_lr]

In [ ]:
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=callbacks)

In [ ]:
def plot_training_history(history):

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(20, 5))

    # Plot training and validation accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

In [ ]:
plot_training_history(history)

In [ ]:
true_classes=test_generator.classes
predicted_classes=np.argmax(model.predict(test_generator,steps=int(np.ceil(test_generator.samples/test_generator.batch_size))),axis=1)
class_label=list(test_generator.class_indices.keys())

cm=confusion_matrix(true_classes,predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_label, yticklabels=class_label)
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# Printing the classification report
report = classification_report(true_classes, predicted_classes, target_names=class_label)
print("Classification Report:\n", report)

In [ ]:
# Emotion classes for the dataset
Emotion_Classes = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']

# Assuming test_generator and model are already defined
batch_size = test_generator.batch_size

# Selecting a random batch from the test generator
Random_batch = np.random.randint(0, len(test_generator) - 1)

# Selecting random image indices from the batch
Random_Img_Index = np.random.randint(0, batch_size, 10)

# Setting up the plot
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 5),
                         subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    # Fetching the random image and its label
    Random_Img = test_generator[Random_batch][0][Random_Img_Index[i]]
    Random_Img_Label = np.argmax(test_generator[Random_batch][1][Random_Img_Index[i]], axis=0)

    # Making a prediction using the model
    Model_Prediction = np.argmax(model.predict(tf.expand_dims(Random_Img, axis=0), verbose=0), axis=1)[0]

    # Displaying the image
    ax.imshow(Random_Img.squeeze(), cmap='gray')  # Assuming the images are grayscale
    # Setting the title with true and predicted labels, colored based on correctness
    color = "green" if Emotion_Classes[Random_Img_Label] == Emotion_Classes[Model_Prediction] else "red"
    ax.set_title(f"True: {Emotion_Classes[Random_Img_Label]}\nPredicted: {Emotion_Classes[Model_Prediction]}", color=color)

plt.tight_layout()
plt.show()

In [ ]:
train_data_dir = '/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/train'
test_data_dir = '/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/test'
val_data_dir='/kaggle/input/lungs-disease-dataset-4-types/Lung Disease Dataset/val'



# Set some parameters
img_width, img_height = 224, 224  # Size of images
batch_size = 64
epochs = 10
num_classes = 5
data_generator = ImageDataGenerator(
                                rescale = 1 / 255.,
                                rotation_range=10,
                                zoom_range=0.2,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                horizontal_flip=True,
                                fill_mode='nearest',
                                # validation_split=0.2
                                )

test_preprocessor = ImageDataGenerator(
    rescale = 1 / 255.,
)


train_generator = data_generator.flow_from_directory(
                                train_data_dir,
                                target_size=(img_width, img_height),
                                batch_size=batch_size,
                                class_mode='categorical',
                                color_mode='rgb',
                                subset='training',
                                shuffle = True)

val_generator = data_generator.flow_from_directory(
                                val_data_dir,
                                target_size=(img_width, img_height),
                                batch_size=batch_size,
                                class_mode='categorical',
                                color_mode='rgb',
                                shuffle = True)

test_generator = test_preprocessor.flow_from_directory(
                                test_data_dir,
                                target_size=(img_width, img_height),
                                batch_size=batch_size,
                                class_mode='categorical',
                                color_mode='rgb',)
                                # subset='validation')

In [ ]:

# Callback for early stopping
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True)

# Callback to reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=6,
                              verbose=1,
                              min_delta=0.0001)

In [ ]:
callbacks=[earlystop,reduce_lr]

In [ ]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Compiling the model
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=callbacks)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.applications import ResNet50V2
from keras.regularizers import l2

# Define the ResNet50V2 model with input shape
def Create_ResNet50V2_Model():
    input_layer = Input(shape=(224, 224, 3))  # Define the input layer explicitly
    resnet_layer = ResNet50V2(input_shape=(224, 224, 3),
                               include_top=False,
                               weights='imagenet')(input_layer)  # Include ResNet50V2 as a layer

    # Add custom layers after ResNet50V2
    x = Dropout(0.25)(resnet_layer)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    # x = Dense(64, activation='relu')(x)
    # x=Dropout(0.1)(x)
    output_layer = Dense(5, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

# Instantiate and summarize the model
model = Create_ResNet50V2_Model()
model.summary()

In [ ]:
# Compiling the model
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=callbacks)

In [ ]:
disease_Classes = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']

# Assuming test_generator and model are already defined
batch_size = test_generator.batch_size

# Selecting a random batch from the test generator
Random_batch = np.random.randint(0, len(test_generator) - 1)

# Selecting random image indices from the batch
Random_Img_Index = np.random.randint(0, batch_size, 10)

# Setting up the plot
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 5),
                         subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    # Fetching the random image and its label
    Random_Img = test_generator[Random_batch][0][Random_Img_Index[i]]
    Random_Img_Label = np.argmax(test_generator[Random_batch][1][Random_Img_Index[i]], axis=0)

    # Making a prediction using the model
    Model_Prediction = np.argmax(model.predict(tf.expand_dims(Random_Img, axis=0), verbose=0), axis=1)[0]

    ax.imshow(Random_Img.squeeze(), cmap='gray')  # Assuming the images are grayscale
    # Setting the title with true and predicted labels, colored based on correctness
    color = "green" if disease_Classes[Random_Img_Label] == disease_Classes[Model_Prediction] else "red"
    ax.set_title(f"True: {disease_Classes[Random_Img_Label]}\nPredicted: {disease_Classes[Model_Prediction]}", color=color)

plt.tight_layout()
plt.show()

In [ ]:
def plot_training_history(history):

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(20, 5))

    # Plot training and validation accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

In [ ]:
true_classes=test_generator.classes
predicted_classes=np.argmax(model.predict(test_generator,steps=int(np.ceil(test_generator.samples/test_generator.batch_size))),axis=1)
class_label=list(test_generator.class_indices.keys())

cm=confusion_matrix(true_classes,predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_label, yticklabels=class_label)
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

def Create_ResNet50V2_Model():
    input_layer = Input(shape=(224, 224, 3))  # Define the input layer explicitly
    resnet_layer = ResNet50V2(input_shape=(224, 224, 3),
                               include_top=False,
                               weights='imagenet')(input_layer)  # Include ResNet50V2 as a layer

    # Add custom layers after ResNet50V2
    x = Dropout(0.25)(resnet_layer)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    output_layer = Dense(5, activation='softmax')(x)  # 5 classes for multiclass classification

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

# Initialize the model
model = Create_ResNet50V2_Model()

# Freeze the ResNet50V2 layers initially (feature extraction phase)
for layer in model.layers[1].layers:
    layer.trainable = False

# Compile the model with a lower learning rate (for feature extraction)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the custom layers only (initial training phase)
history = model.fit(
    train_generator,  # Assuming you have a data generator for training
    epochs=10,        # Train for a few epochs (e.g., 10 epochs)
    validation_data=val_generator,  # Validation data
    callbacks=callbacks
)




In [ ]:
disease_Classes = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']

# Assuming test_generator and model are already defined
batch_size = test_generator.batch_size

# Selecting a random batch from the test generator
Random_batch = np.random.randint(0, len(test_generator) - 1)

# Selecting random image indices from the batch
Random_Img_Index = np.random.randint(0, batch_size, 10)

# Setting up the plot
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10),
                         subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    # Fetching the random image and its label
    Random_Img = test_generator[Random_batch][0][Random_Img_Index[i]]
    Random_Img_Label = np.argmax(test_generator[Random_batch][1][Random_Img_Index[i]], axis=0)

    # Making a prediction using the model
    Model_Prediction = np.argmax(model.predict(tf.expand_dims(Random_Img, axis=0), verbose=0), axis=1)[0]

    # Set title color based on prediction correctness
    color = "green" if disease_Classes[Random_Img_Label] == disease_Classes[Model_Prediction] else "red"

    ax.imshow(Random_Img.squeeze(), cmap='gray')  # Assuming the images are grayscale
    ax.set_title(f"True: {disease_Classes[Random_Img_Label]}\nPredicted: {disease_Classes[Model_Prediction]}", color=color)

plt.tight_layout()
plt.show()




In [ ]:
def plot_training_history(history):

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(20, 5))

    # Plot training and validation accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

In [ ]:
plot_training_history(history)

In [ ]:
true_classes=test_generator.classes
predicted_classes=np.argmax(model.predict(test_generator,steps=int(np.ceil(test_generator.samples/test_generator.batch_size))),axis=1)
class_label=list(test_generator.class_indices.keys())

cm=confusion_matrix(true_classes,predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_label, yticklabels=class_label)
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
model.save('best_model.h5')

In [ ]:
import gradio as gr
import numpy as np
from keras.models import load_model
import cv2

# Load your trained model
model = load_model('best_model.h5')  # Replace with your saved model path

# List of class names
Emotion_Classes = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']

# Define a prediction function
def predict_condition(image):
    # Resize the image to match the model's expected input size
    image = cv2.resize(image, (224, 224))  # Adjust (224, 224) if your model expects a different size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image / 255.0  # Normalize the image if your model expects it

    # Predict the class
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)

    return f"Predicted Condition: {Emotion_Classes[predicted_class]}"

# Define Gradio interface
interface = gr.Interface(
    fn=predict_condition,
    inputs=gr.Image(),  # No shape argument needed here
    outputs=gr.Text(),
    title="Disease Classification from Chest X-Ray",
    description="Upload a chest X-ray image, and the model will predict the disease category."
)

# Launch the interface
interface.launch()